In [1]:
import requests
import json
import datetime
import pandas as pd

In [2]:
def r_to_j(r):
    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
#    print(json.dumps(myJ_s, indent=1))
    return myJ


In [3]:
def do_r(req, page_size=2000, page_num=0,debug=False, attrs=[]):
    myJ = {}
    rJ = {'count': 0, 'g' : {}}
    keepGoing=1
    gDebug=1

    ctr=1
    
    req += '&page_size='+str(page_size)+'&page_num='

    while keepGoing:
        keepGoing=1
        page_num += 1
        if page_num > 599:
            keepGoing = 0

        ctr += 1
        if ctr >= 100:
            print ('100th request', datetime.datetime.now())
            ctr=1
            
        r = requests.get(req + str(page_num) )
        
        if debug:
            print(r, page_num, datetime.datetime.now())
        
        myJ = r_to_j(r)
        
        if debug:
            print('myj=', json.dumps(myJ, indent=1))
            debug=0
        
        if len(myJ['items']) == 0:
            keepGoing=0
            continue

        rJ['count']+= len(myJ['items'])
        outJ={}
        for j in myJ['items']:
            if gDebug:
                print (json.dumps(j['umm']))
                if 'GranuleUR' in j['umm']:
                    print('****************--->', j)
                else:
                    print ('producer_granule_id NOT FOUND!', j)
                gDebug=0
            outJ={}
            for a in attrs:
                if a in j['umm']:
                    outJ[a]=j['umm'][a]
                
            rJ['g'][j['umm']['GranuleUR']]=outJ

    return rJ

In [4]:
# Get a list of short_names from a provider
import time

my_provider='PODAAC'
my_shortNames=[]
#print (r.headers)
myJ_s = ''
collectionsJ={}
uniqueKeysJ={}

page_num=0
keepGoing=1
debug=0
while keepGoing:
    
    page_num += 1
    if page_num > 200:
        keepGoing = 0
    print ('before request.get')  
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?platform=AQUA&sort_key=short_name&page_size=2&page_num='+str(page_num))
    r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?provider=PODAAC&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?instrument=MODIS&INSTRUMENT=MODIS&provider=LPDAAC_ECS&sort_key=short_name&page_size=2&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?granule_data_format=HDF-EOS2&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    print ('after request.get')  

    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
    #print(myJ_s)
    myJ = json.loads(myJ_s)
    #print(json.dumps(myJ, indent=1))

    if len(myJ['items']) == 0:
        keepGoing=0
        continue
    
    for r in myJ['items']:
        j=r['meta']
        #print ('j=', j)
        if debug == 1:
            print ('r=', json.dumps(r,indent=1))
            debug=0
        #for k in j:
        #    print ('meta',k, j[k])
        j=r['umm']
#        for k in j:
            #print ('umm', k)
#            if k == 'ShortName':
        my_shortNames.append({'ShortName' : j['ShortName'],
                                     'native-id' : r['meta']['native-id'],
                                'EntryTitle' : j['EntryTitle'],
                              'Count':r['meta']['granule-count'] if 'granule-count' in r['meta'] else 0,
                             'Format' : j ['ArchiveAndDistributionInformation'] if 'ArchiveAndDistributionInformation' in j else 'UNKNOWN'
                             })

i=1
for s in my_shortNames:
    print (i, s)
    i+=1
#print (my_shortNames)

before request.get
after request.get
before request.get
after request.get
1 {'ShortName': 'ABI_G16-STAR-L2P-v2.70', 'native-id': 'GHRSST+NOAA/STAR+GOES-16+ABI+L2P+America+Region+SST+v2.70+dataset+in+GDS2', 'EntryTitle': 'GHRSST NOAA/STAR GOES-16 ABI L2P America Region SST v2.70 dataset in GDS2', 'Count': 0, 'Format': {'FileDistributionInformation': [{'Format': 'NETCDF', 'FormatType': 'Native'}]}}
2 {'ShortName': 'ABI_G16-STAR-L3C-v2.70', 'native-id': 'GHRSST+NOAA/STAR+GOES-16+ABI++L3C+America+Region+SST+v2.70+dataset+in+GDS2', 'EntryTitle': 'GHRSST NOAA/STAR GOES-16 ABI  L3C America Region SST v2.70 dataset in GDS2', 'Count': 0, 'Format': {'FileDistributionInformation': [{'Format': 'NETCDF', 'FormatType': 'Native'}]}}
3 {'ShortName': 'ABI_G17-STAR-L2P-v2.71', 'native-id': 'GHRSST+NOAA/STAR+GOES-17+ABI+L2P+America+Region+SST+v2.71+dataset+in+GDS2', 'EntryTitle': 'GHRSST NOAA/STAR GOES-17 ABI L2P America Region SST v2.71 dataset in GDS2', 'Count': 0, 'Format': {'FileDistributionInformati

In [6]:
r_base = 'https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name='
i=1


file=open('podaac_l2p_urls.txt','w')

for s in my_shortNames:
#    print (s)
    if s['ShortName']=='ABI_G16-STAR-L2P-v2.70':
        pass
    else:
        continue

    
# {'ShortName': 'AA_L2A', 
#  'native-id': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001', 
#  'EntryTitle': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001',
#  'Count': 0, 
#  'Format': {'FileDistributionInformation': [{'FormatType': 'Native', 'Format': 'HDF-EOS', 'FormatDescription': 'FTP 68 MB'}]}}
    
    r = r_base + s['ShortName']
    r += '&temporal=2021-08-01T00:00:00Z,2021-09-02T23:59:59Z'
    
    print (r)

    gJ = do_r(r, attrs=['GranuleUR','DataGranule', 'RelatedUrls', 'SpatialExtent', 'TemporalExtent'],debug=False)
    
    if gJ['count'] == 0:
        continue
        
    print ('# Granules', gJ['count'])
    
    for g in gJ['g']:
        print ('Example G', json.dumps(gJ['g'][g]))
        break
    
    gn=0
    granuleSizeTotal=0.0
    
    #print (i, 'Collection:', s['ShortName'], s['Count'], s)
    i+=1
    for g in gJ['g']:
        gn+=1
        gm=gJ['g'][g]
        #print ('g dump')
        #print(g, json.dumps(gJ['g'][g], indent=1))
        
        g_urls = { 'get' : None, 'opendap' : None}
        for u in gm['RelatedUrls']:
            #
            # RelatedURLS inconsistent within same collection!!!!!!!!
            #
            if 'opendap' not in u['URL']:
                g_urls['get'] = u['URL']
            if 'opendap' in u['URL']:
                g_urls['opendap'] = u['URL']
        
        if g_urls['get'] == None or g_urls['opendap'] == None:
            print ('wft?', gm)
        else:
            x_line = s['ShortName']
            x_line += '~' + g 
            x_line += '~' + gm['TemporalExtent']['RangeDateTime']['BeginningDateTime'] 
            x_line += '~' + gm['TemporalExtent']['RangeDateTime']['EndingDateTime'] 
            x_line += '~' + str(gm['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['BoundingRectangles'][0]['WestBoundingCoordinate'])
            x_line += '~' + str(gm['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['BoundingRectangles'][0]['SouthBoundingCoordinate'])
            x_line += '~' + str(gm['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['BoundingRectangles'][0]['EastBoundingCoordinate'])
            x_line += '~' + str(gm['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['BoundingRectangles'][0]['NorthBoundingCoordinate'])
            x_line += '~' + g_urls['get']
            x_line += '~' + g_urls['opendap']
            x_line += '\n'
            file.write(x_line)

        #print (x_line)

#"RelatedUrls": [
#    {"URL": "https://podaac-tools.jpl.nasa.gov/drive/files/allData/ghrsst/data/GDS2/L2P/GOES16/STAR/v2.70/2017/349/20171215000000-STAR-L2P_GHRSST-SSTsubskin-ABI_G16-ACSPO_V2.70-v02.0-fv01.0.nc",
#     "Type": "GET DATA", 
#     "Description": "The HTTP location for the granule."
#    }, 
#    {"URL": "https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L2P/GOES16/STAR/v2.70/2017/349/20171215000000-STAR-L2P_GHRSST-SSTsubskin-ABI_G16-ACSPO_V2.70-v02.0-fv01.0.nc.html",
#     "Type": "USE SERVICE API",
#     'Type': 'USE SERVICE API', 
#     "Subtype": "OPENDAP DATA",
#     'Subtype': 'OPENDAP DATA', 
#     "Description": "The OPENDAP location for the granule.", "MimeType": "text/html"
#    }
#    ]

#what the fuck? 
#[
#  {'URL': 'https://podaac-tools.jpl.nasa.gov/drive/files/allData/ghrsst/data/GDS2/L2P/GOES16/STAR/v2.70/2017/349/20171215000000-STAR-L2P_GHRSST-SSTsubskin-ABI_G16-ACSPO_V2.70-v02.0-fv01.0.nc', 
#   'Type': 'GET DATA', 
#   'Description': 'The HTTP location for the granule.'
#  }, 
#{'URL': 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L2P/GOES16/STAR/v2.70/2017/349/20171215000000-STAR-L2P_GHRSST-SSTsubskin-ABI_G16-ACSPO_V2.70-v02.0-fv01.0.nc.html', 
#   'Type': 'USE SERVICE API', 
#   'Subtype': 'OPENDAP DATA', 
#   'Description': 'The OPENDAP location for the granule.', 'MimeType': 'text/html'}
#]

# "SpatialExtent": {
#  "HorizontalSpatialDomain": {
#   "Geometry": {
#    "BoundingRectangles": [
#     {
#      "WestBoundingCoordinate": -156.195,
#      "EastBoundingCoordinate": 6.199,
#      "NorthBoundingCoordinate": 81.147,
#      "SouthBoundingCoordinate": -81.149
#     }
#    ]
#   }
#  }
# },                

#for i in cmr_granule_table:
#    print (i)
#    break

file.close()


https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name=ABI_G16-STAR-L2P-v2.70&temporal=2021-08-01T00:00:00Z,2021-09-02T23:59:59Z
{"TemporalExtent": {"RangeDateTime": {"EndingDateTime": "2021-08-01T00:09:53.000Z", "BeginningDateTime": "2021-08-01T00:00:22.000Z"}}, "MetadataSpecification": {"URL": "https://cdn.earthdata.nasa.gov/umm/granule/v1.6.4", "Name": "UMM-G", "Version": "1.6.4"}, "GranuleUR": "20210801000000-STAR-L2P_GHRSST-SSTsubskin-ABI_G16-ACSPO_V2.70-v02.0-fv01.0", "ProviderDates": [{"Type": "Insert", "Date": "2021-08-26T02:26:28.400Z"}, {"Type": "Update", "Date": "2021-08-26T02:26:28.400Z"}], "SpatialExtent": {"HorizontalSpatialDomain": {"Geometry": {"BoundingRectangles": [{"WestBoundingCoordinate": -156.195, "SouthBoundingCoordinate": -81.149, "EastBoundingCoordinate": 6.199, "NorthBoundingCoordinate": 81.147}]}}}, "DataGranule": {"ArchiveAndDistributionInformation": [{"SizeUnit": "MB", "Size": 41.810173988342285, "Checksum": {"Value": "e5d9bc597de0c9af344c73b698

CREATE EXTERNAL TABLE `cmr_3`(
 GranuleUR string,
 DataGranule string,
 CollectionReference string,
 TemporalExtent string,
 SpatialExtent string
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'quoteChar'='\u0000', 
  'separatorChar'='~') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://nasa-ems-sandbox/ems_analysis/ad-hoc/analyst1/cmr_3'
TBLPROPERTIES (
  'skip.header.line.count'='1', 
  'transient_lastDdlTime'='1620856378')